In this assignment I will explore and cluster the neigbourhoods in Toronto. In this first assignment of this whole assignment the goal is to get the data from  Wikipedia page, transform it to a pandas dataframe and clean it so that the data is in a structured format. Lets begin.

In [4]:
# at first i import needed libraries 
import pandas as pd
import numpy as np


In [5]:
# then I wrangle the data from Wikibedia page
data_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto_data = pd.read_html(data_path)


After I wrangled the data I checked in what format the data is (result is a list) and how many datasets does the data include (result was that the lenght of the dataset is 3). I also checked is the list on what format, and the result of this was a pandas Dataframe. 

So no I need to transform the first list to a dataframe called df_toronto_data. I also checked that does the data include duplicates in Postal Codes or Borough or Neighborhood - and the result is no. I will also clean the celss with borought such as Not assigned.

In [6]:
#trnasforming the data form first list into a Dataframe
df_toronto_data = toronto_data[0]

In [8]:
df_toronto_data = df_toronto_data.drop(df_toronto_data.loc[df_toronto_data["Borough"]=="Not assigned"].index, inplace=False).reset_index(drop = True)
df_toronto_data.head(12)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Created Dataframe consist three colums: PostalCode, Borough and Neighborhood
I have now ignored celss with a borought that is Not assingned.
In the data there is no twice listed PostalCodes - Neighborhoods which have the same PostalCode are mentioned in the same PostalCode (rows are combined into one row, separated with a comma)
Cells do not have Not assingned text either in Borough or in Neigborhood.


In [9]:
print(df_toronto_data.shape)

(103, 3)


In this second assignment the mission is to built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. Because the given package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, I will use this link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data.

So now I need to read the csv file and then merge the DataFrame whit df_toronto_data. The result will be a new DataFrame including information and data drom both datasets.

In [19]:
# at first I read the csv file
data_path = "http://cocl.us/Geospatial_data"
geogordinates_toronto = pd.read_csv(data_path)

After I readed the file I checked the len of the file (103), type of the file (pandas.core.frame.DataFrame) and shape of the file ((103, 3). Data is same type as the previos and the shape of the file is the same than in df_toroto_data so I can go on and unite these to datasets.

In [20]:
# uniting these two datasets
df_data_toronto = df_toronto_data.merge(geogordinates_toronto, left_on="Postal Code", right_on="Postal Code")
df_data_toronto.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [21]:
print(df_data_toronto.shape)

(103, 5)
